# Continuous Control of Reacher

---

This notebook is the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Packages needed

* [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) 
* [NumPy](http://www.numpy.org/)
* [pytorch](https://pytorch.org/)
* [plotly](https://plot.ly/)

In [1]:
import time
from collections import deque

import numpy as np
import torch

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import pandas as pd

init_notebook_mode(connected=True)


from unityagents import UnityEnvironment

env = UnityEnvironment(file_name='Reacher.app', no_graphics=True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [2]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [3]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### Train Agent: Script

In [4]:
from ddpg_agent import Agent
agent = Agent(state_size=state_size, action_size=action_size, random_seed=2)

Actor(
  (fc1): Linear(in_features=33, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
)
Actor(
  (fc1): Linear(in_features=33, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
)
Critic(
  (fcs1): Linear(in_features=33, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=132, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-0

In [5]:
n_episodes=1000

In [6]:
scores = []

def train_ddpg(n_episodes=n_episodes, scores=scores, load_checkpoints=False):
    if load_checkpoints:
        agent.load_checkpoints()
        
    scores_window = deque(maxlen=100)

    for episode in range(n_episodes):
        
        env_info = env.reset(train_mode=True)[brain_name]            # reset the environment
        states = env_info.vector_observations
        agent.reset()                                                # reset the agent
        score = np.zeros(num_agents)
        
        while True:
            actions = agent.act(states)        
            env_info = env.step(actions)[brain_name]                 # send the action to the environment                            
            next_states = env_info.vector_observations               # get the next state        
            rewards = env_info.rewards                               # get the reward        
            dones = env_info.local_done                              # see if episode has finished        
            agent.step(states, actions, rewards, next_states, dones)

            score += rewards                                         # update the score        
            states = next_states                                     # roll over the state to next time step                                                                
            if np.any(dones):                                        # exit loop if episode finished        
                break                                        

        agent.save_checkpoints()
        scores.append(np.mean(score))
        scores_window.append(np.mean(score))
        print('\rEpisode: {} Average Window Score: {:.4f} \tPresent Score: {:.2f} \tMax Window Score: {:.2f}'.format(
            episode, np.mean(scores_window), np.mean(score),  np.max(scores_window)), end="")  
        
        if np.mean(scores_window) > 30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Window Score: {:.2f}'.format(
                episode, np.mean(scores_window)))
            break    

### Train the agent

In [ ]:
train_ddpg(load_checkpoints=False)

Episode: 57 Average Window Score: 0.0071 	Present Score: 0.00 	Max Window Score: 0.12

In [ ]:
data = [go.Scatter(x=np.arange(len(scores)), y=scores)]
layout = dict(
    title = 'Improvement of scores during trainig',
    xaxis = dict(title = 'Number of Episodes'),
    yaxis = dict(title = 'Score'),
)

fig = dict(data=data, layout=layout)
iplot(fig)

In [ ]:
env.close()